# Loading prereqs and data

In [1]:
import sys
sys.path.append("../")
from utils import *
import snap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from Queue import Queue
from collections import defaultdict

railroad_map_path = "../generate_railroad_map/railroad_map.txt"
stations_json_path = "../generate_railroad_map/"

# Notice - all the nodes (stations) in this project is 1-indexed
G, N, M, weight = load_map(railroad_map_path)
station_to_index_map, train_to_line_map, station_to_gps_map = load_stations_and_trains_and_gps_json(stations_json_path)
train_importance = get_train_importance(train_to_line_map)
index_to_station_map = {}
for station in station_to_index_map:
    c = station_to_index_map[station]
    index_to_station_map[c] = station

with open("railroad_effect.bin", "r") as f:
    railroad_effect = pickle.load(f)

with open("compressed_effect.bin", "r") as f:
    compressed_effect = pickle.load(f)

with open("railmap.bin", "r") as f:
    railmap = pickle.load(f)

# Compressed Graph

In [2]:
print(len(compressed_effect))

2053


In [3]:
nbrs = defaultdict(list)
for s1, s2, _ in compressed_effect:
    nbrs[s1].append(s2)
    nbrs[s2].append(s1)

print(len(nbrs))

degs = [len(nbrs[s]) for s in nbrs]
plt.clf()
plt.title("Degree distribution of all stations, compressed")
plt.xlabel("degree")
plt.ylabel("# of stations")
plt.hist(degs, bins=11)
plt.savefig("degree_distribution")

1222


AttributeError: 'module' object has no attribute 'to_rgba'

In [4]:
rails = []
imps = []
for rail in compressed_effect:
    rails.append(rail)
    imps.append(sum([train_importance[train] for train in compressed_effect[rail]]))

ind = np.argsort(imps)[::-1]

In [26]:
def increm_potential(s1, s2):
    return sum([train_importance[train] for train in (s1 - s2)])

sims = np.zeros((2053, 2053))
for i in range(2053):
    for j in range(2053):
        sim = increm_potential(compressed_effect[rails[ind[i]]], compressed_effect[rails[ind[j]]])
        sims[i][j] = sim

plt.imshow(sims, cmap='hot')

In [6]:
plt.savefig('railroad_heatmap')

AttributeError: 'module' object has no attribute 'to_rgba'

In [27]:
np.save('sims.npy', sims)

In [17]:
plt.clf()
x = np.arange(2053)
y = sorted(imps)[::-1]
plt.plot(x, y)

AttributeError: 'module' object has no attribute 'to_rgba'

In [18]:
plt.savefig('importance')

AttributeError: 'module' object has no attribute 'to_rgba'

In [19]:
np.save('imp', y)